In [42]:
import builtins
import copy

from selenium import webdriver
from selenium.common import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.common import StaleElementReferenceException

from itertools import combinations
from datetime import datetime

import pandas as pd
import numpy as np

import time

In [43]:
def print(*args, **kwargs):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    builtins.print(f"[{timestamp}] ", *args, **kwargs)

def get_timestamp():
    return str(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [44]:
EDGE_DRIVER_PATH = "edgedriver_win32/msedgedriver.exe"
driver = webdriver.Edge(service=Service(EDGE_DRIVER_PATH))

In [45]:
wait = WebDriverWait(driver, 10)

In [46]:
driver.get("https://rustypot.com/coinflip")

In [47]:
def get_possible_bet_values(inventory_items_values, max_items=3):
    print(f"there are {len(inventory_items_values)} items in inventory with total value: {sum(inventory_items_values):.2f}$")
    possible_bet_values = {}
    for r in range(1, max_items + 1):
        for combo in combinations(inventory_items_values, r):
            s = sum(combo)
            max_tax = s * 2 * 0.05
            good_combo = True
            for item_value in combo:
                if item_value < max_tax:
                    good_combo = False
            if good_combo:
                possible_bet_values[s] = list(combo)
    possible_bet_values_sorted = dict(sorted(possible_bet_values.items()))
    
    return possible_bet_values_sorted

In [48]:
def get_inventory_items():
    create_a_game_buttom = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "createCoinflipButton"))
    )
    create_a_game_buttom.click()
    
    try:
        inventory_items = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "InventoryItem"))
        )
        
        # inventory_items = inventory_items_div.find_elements(By.CLASS_NAME, "InventoryItem")
        inventory_items_values = []
        for inventory_item in inventory_items:
            inventory_items_values.append(float(inventory_item.text.split("$")[1]))
        
        driver.get("https://rustypot.com/coinflip")

        print(f"inventory items gathered: {inventory_items_values}", )
        
        return inventory_items_values
    except TimeoutException:
        print(f"Timed out waiting for inventory items")
        driver.get("https://rustypot.com/coinflip")
        return get_inventory_items()

In [49]:
inventory_items_values = get_inventory_items()
# inventory_items_values = [62.31, 31.75, 31.75, 20.86, 17.38, 13.34, 13.34] ### TEST WITH HARDCODED ITEMS
start = time.time()
possible_bet_values = get_possible_bet_values(inventory_items_values)
end = time.time()

print(f"{end - start:.2f} seconds")
possible_bet_values

[2025-03-24 00:02:09]  inventory items gathered: [7.01, 5.22, 5.21, 4.94, 4.86, 4.39, 4.29, 4.29, 2.95, 2.95, 2.81, 2.81, 2.79, 2.21, 1.99, 1.69, 1.68, 1.67, 1.67, 1.65, 1.64, 1.64, 1.64, 1.64, 1.63, 1.58, 1.58, 1.58, 1.58, 1.58, 1.58, 1.58, 1.58, 1.55, 1.54, 1.53, 1.52, 1.52, 1.52, 1.51, 1.51, 1.51, 1.51, 1.49, 1.49, 1.49, 1.49, 1.48, 1.48, 1.46, 1.45, 1.45, 1.44, 1.42, 1.41, 1.41, 1.41, 1.41, 1.39, 1.38, 1.38, 1.37, 1.37, 1.32, 1.32, 1.32, 1.26, 1.23, 1.23, 1.23, 1.23, 1.23, 1.23, 1.23, 1.22, 1.21, 1.21, 1.12, 1.08, 1.04, 1.01, 1.01, 1.0, 0.99, 0.97, 0.97, 0.97, 0.96, 0.94, 0.94, 0.93, 0.91, 0.9, 0.9, 0.9, 0.89, 0.88, 0.88, 0.87, 0.87, 0.86, 0.86, 0.86, 0.85, 0.85, 0.85, 0.85, 0.84, 0.84, 0.84, 0.84, 0.82, 0.8, 0.8, 0.8, 0.79, 0.78, 0.77, 0.77, 0.77, 0.76, 0.76, 0.75, 0.72, 0.71, 0.66, 0.66, 0.66, 0.66]
[2025-03-24 00:02:09]  there are 129 items in inventory with total value: 193.98$
[2025-03-24 00:02:09]  0.12 seconds


{0.66: [0.66],
 0.71: [0.71],
 0.72: [0.72],
 0.75: [0.75],
 0.76: [0.76],
 0.77: [0.77],
 0.78: [0.78],
 0.79: [0.79],
 0.8: [0.8],
 0.82: [0.82],
 0.84: [0.84],
 0.85: [0.85],
 0.86: [0.86],
 0.87: [0.87],
 0.88: [0.88],
 0.89: [0.89],
 0.9: [0.9],
 0.91: [0.91],
 0.93: [0.93],
 0.94: [0.94],
 0.96: [0.96],
 0.97: [0.97],
 0.99: [0.99],
 1.0: [1.0],
 1.01: [1.01],
 1.04: [1.04],
 1.08: [1.08],
 1.12: [1.12],
 1.21: [1.21],
 1.22: [1.22],
 1.23: [1.23],
 1.26: [1.26],
 1.32: [0.66, 0.66],
 1.37: [0.71, 0.66],
 1.38: [0.72, 0.66],
 1.39: [1.39],
 1.41: [1.41],
 1.4100000000000001: [0.75, 0.66],
 1.42: [0.76, 0.66],
 1.43: [0.72, 0.71],
 1.4300000000000002: [0.77, 0.66],
 1.44: [0.78, 0.66],
 1.45: [1.45],
 1.4500000000000002: [0.79, 0.66],
 1.46: [0.75, 0.71],
 1.47: [0.75, 0.72],
 1.48: [0.76, 0.72],
 1.49: [0.77, 0.72],
 1.5: [0.78, 0.72],
 1.51: [0.76, 0.75],
 1.52: [0.76, 0.76],
 1.5299999999999998: [0.82, 0.71],
 1.53: [0.77, 0.76],
 1.54: [0.77, 0.77],
 1.5499999999999998: [0.84,

In [50]:
def get_bet_items(bet_values, coinflip_value):
    for bet_value in bet_values:
        if (coinflip_value * 0.9 < bet_value <= coinflip_value * max_bet_percentage and 
                # coinflip_value not in bets and 
                min_bet_value < coinflip_value < max_bet_value):
            return bet_values[bet_value]
    
    return None


def join_coinflip(join_button):

    try:
        join_button.click()
        
        spinner_locator = (By.CLASS_NAME, "spinner")
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located(spinner_locator)
        )
    except TimeoutException:
        print(f"Inventory loading timed out during selecting bet items")
        driver.get("https://rustypot.com/coinflip")
        join_coinflip(join_button)
    
    
def select_bet_items(bet_items_values):
    inventory_items_locator = (By.CLASS_NAME, "InventoryItem")
    inventory_items = WebDriverWait(driver, 3).until(
        EC.presence_of_all_elements_located(inventory_items_locator)
    )
    
    for inventory_item in inventory_items:
        item_value = float(inventory_item.text.split("$")[1])
        if item_value in bet_items_values:
            inventory_item.click()
            bet_items_values.remove(item_value)
    

def press_deposit_button():
    deposit_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[@onclick='requestCfDeposit()']"))
    )
    deposit_button.click()
    
    
def press_accept_deposit_button_site():
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@onclick='closeAcceptDeposit()']"))
    )
    accept_button.click()

def press_accept_winnings_button_site():
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH,
                                "//button[@onclick='closeAcceptWinnings()']"))
    )
    accept_button.click()

def press_no_doubledown_button():
    cancel_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@id='DDCancel' and contains(@class, 'DDOption')]"))
    )
    cancel_button.click()


def accept_steam_trade(deposit, acceptWinning):
    
    # 1.open the new window
    WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
                
    original_window = driver.current_window_handle
    new_window = [w for w in driver.window_handles if w != original_window][0]
    driver.switch_to.window(new_window)
    
    
    # 2.close if profile reported    
    try:
        warning_element = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((
                By.XPATH,
                "//*[contains(., 'have reported Blown for attempting trade scams')]"
            ))
        )
        ok_button = WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((
                By.XPATH,
                "//div[contains(@class, 'btn_grey_steamui') and span[text()='OK']]"
                ))
        )
        ok_button.click()
    except TimeoutException:
        pass
    
    # 3. Accept the trade
    confirm_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
    )
    confirm_button.click()
    time.sleep(0.25)
    
    if deposit:
        confirm_gift_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[span[text()='Yes, this is a gift']]"))
        )
        confirm_gift_button.click()
        time.sleep(0.25)
    elif acceptWinning:
        confirm_gift_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='btn_green_steamui btn_medium']"))
        )
        confirm_gift_button.click()
        time.sleep(0.25)
    
    accept_trade_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "trade_confirmbtn"))
    )
    accept_trade_btn.click()
    time.sleep(0.5)

    driver.close()
    driver.switch_to.window(original_window)
    
    
def open_cf_window_and_wait_to_flip():
    view_button = coinflip_div.find_element(By.XPATH, ".//button[text()='View']")
    view_button.click()
    
    # Wait until the <p> inside #fliper-coin disappears
    try:
        p_element = driver.find_element(By.CSS_SELECTOR, "#fliper-coin > p")
        WebDriverWait(driver, 120).until(EC.staleness_of(p_element))
    except:
        # If <p> is already gone, just wait for the new state
        pass
        
    
    WebDriverWait(driver, 120).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#fliper-coin .flipper"))
    )


def get_winner():
    # Wait for the parent div to appear
    creator_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "creator-imgs")))
    
    # Wait for the child img.pick within that div
    color_img = WebDriverWait(driver, 10).until(lambda d: creator_div.find_element(By.XPATH, ".//img[@class='pick']"))
    
    # Get the src
    src = color_img.get_attribute("src")
    
     # Extract the color from the filename
    if "Red" in src:
        creator_color = "Red"
    elif "Black" in src:
        creator_color = "Black"
    else:  # creator_color = "Unknown"
        time.sleep(0.5)
        print(f"creator_color is {src}. Retry getting creator")
        return get_winner()
    
    # 1. Get the 'flipper' element
    flipper = driver.find_element(By.CLASS_NAME, "flipper")
    
    # 2. Get the 'transform' style attribute
    transform_style = flipper.get_attribute("style")  # e.g. "transform: rotateY(1620deg);"
    
    # 3. Extract the degree
    import re
    match = re.search(r'rotateY\((\d+)deg\)', transform_style)
    if match:
        degrees = int(match.group(1))
        # Normalize the rotation within 360°
        final_angle = degrees % 360
    
        # Every 180° flips the face, so even number of 180s = same side
        flips = (degrees // 180) % 2
    
        flip_face = "Red" if flips == 1 else "Black"
        print("Coin landed on:", flip_face)
    else:
        time.sleep(0.5)
        print("Couldn't determine coin face. Retry getting flipping face")
        return get_winner()
        
    return creator_color, flip_face

def get_coinflip_items_values(coinflip_items):
    img_elements = coinflip_items.find_elements(By.TAG_NAME, "img")
    coinflip_items_values = []
    for img in img_elements:
        title = img.get_attribute("data-original-title")
        if "|" in title:
            price_str = title.split("$")[-1]
            price = float(price_str)
            coinflip_items_values.append(price)
    
    return coinflip_items_values

def log_bet(bet_value, won_value, win, profit, inventory_value):
    f = open("logs.csv", "a")
    f.write(f"{get_timestamp()},"
            f"{bet_value:.2f},"
            f"{won_value:.2f},"
            f"{win},"
            f"{profit:.2f},"
            f"{inventory_value:.2f}\n")
    f.close()

In [55]:
max_bet_percentage = 0.93
tax_procentage = 0.05
min_bet_value = 1.0
max_bet_value = 4 * (1 + (1 - max_bet_percentage))
# max_bet_value = 4

In [68]:
inventory_items_values = get_inventory_items()
possible_bet_values = get_possible_bet_values(inventory_items_values)

while True:
    try:
        active_coinflips = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ActiveCoinflips")))
        coinflip_row_divs = active_coinflips.find_elements(By.CLASS_NAME, "coinflip")
        
        for coinflip_div in coinflip_row_divs:
            try:
                coinflip_value = float(coinflip_div.get_attribute("coinflip-value"))
                coinflip_items = coinflip_div.find_element(By.CLASS_NAME, "coinflipGameItems")
            except StaleElementReferenceException:
                print(f"coinflip divs are stale. Retry getting coinflip rows")
                break
            
            # cf value + bet value ( max 95% of cf value)
            tax_value = (coinflip_value + coinflip_value * max_bet_percentage) * tax_procentage  # the taxed amount is the total amount of the coinflip
    
            if "items" in coinflip_items.text:
                # print("more than 5 items...")
                continue
    
            coinflip_items_values = get_coinflip_items_values(coinflip_items)
    
            if any(tax_value > item_value for item_value in coinflip_items_values):
                # print(f"not profitable")
                continue
            
            
            # to join a cf, i need to deposit minimum value
            # 1. get the bet items
            bet_items_values = get_bet_items(possible_bet_values, coinflip_value)
            
            if bet_items_values is None:
                # no available bet for this cf based on our items
                continue
            
            ### Press join on available coinflip. if it is ongoing (only View button visible, skip)
            join_button = coinflip_div.find_element(By.TAG_NAME, "button")
            if "Join" not in join_button.text:
                continue
                
            print(f"bet_value = {sum(bet_items_values):.2f} -> bet_items_value{bet_items_values=}")
    
            join_coinflip(join_button)
            
            try:
                select_bet_items(copy.deepcopy(bet_items_values))
            except TimeoutException:
                print(f"Timed out when selecting bet items")
                driver.get("https://rustypot.com/coinflip")
                time.sleep(0.5)
                break
                
            press_deposit_button()
            
            try:
                # check if game already in progress
                element = driver.find_element(By.XPATH, '//div[@id="AcceptDepositBody"]/h4[text()="This game is already in progress!"]')

                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located(element)
                )
                
                # if present, break and continue
                driver.get("https://rustypot.com/coinflip")
                time.sleep(0.5)
                break
            except Exception:
                pass
            
            press_accept_deposit_button_site()
            
            accept_steam_trade(deposit=True, acceptWinning=False)
    
     
            print(f"{coinflip_value=} -> {coinflip_items_values=}")
                
            # bets.append(coinflip_value)
            
            open_cf_window_and_wait_to_flip()
    
            creator_color, flip_face = get_winner()
    
            # extract flip color
            
            if creator_color == flip_face:
                print(f"flip was lost!")
                time.sleep(4) # wait to see the flip
    
                driver.get("https://rustypot.com/coinflip")
                time.sleep(1)
                for item in bet_items_values:
                    inventory_items_values.remove(item)
                    
                log_bet(sum(bet_items_values), coinflip_value, 0, -sum(bet_items_values), sum(inventory_items_values))
                
                possible_bet_values = get_possible_bet_values(inventory_items_values)
                
            else:
                print(f"flip was won!")
                time.sleep(4)
                
                press_no_doubledown_button()
    
                press_accept_winnings_button_site()
    
                accept_steam_trade(deposit=False, acceptWinning=True)
                
                for item in coinflip_items_values:
                    inventory_items_values.append(item)
                    
                log_bet(sum(bet_items_values), coinflip_value, 1, coinflip_value, sum(inventory_items_values))
    
                possible_bet_values = get_possible_bet_values(inventory_items_values)
        
            print(f"Back to searching")
    except StaleElementReferenceException:
        print(f"stale element error -> maybe captch / cf already started / something")
        driver.get("https://rustypot.com/coinflip")

    time.sleep(1)

[2025-03-24 00:19:48]  inventory items gathered: [7.01, 5.24, 5.24, 4.94, 4.86, 4.38, 4.21, 4.21, 3.1, 2.98, 2.97, 2.79, 2.79, 2.79, 2.59, 2.22, 1.99, 1.93, 1.76, 1.7, 1.69, 1.67, 1.66, 1.66, 1.65, 1.62, 1.62, 1.62, 1.62, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.54, 1.53, 1.53, 1.53, 1.53, 1.52, 1.51, 1.51, 1.49, 1.49, 1.49, 1.47, 1.46, 1.46, 1.45, 1.45, 1.43, 1.43, 1.43, 1.43, 1.41, 1.41, 1.41, 1.4, 1.4, 1.38, 1.37, 1.37, 1.37, 1.32, 1.32, 1.31, 1.26, 1.24, 1.24, 1.24, 1.24, 1.24, 1.23, 1.23, 1.23, 1.22, 1.21, 1.21, 1.11, 1.06, 1.06, 1.05, 1.05, 1.0, 0.99, 0.97, 0.96, 0.96, 0.96, 0.94, 0.94, 0.94, 0.93, 0.91, 0.91, 0.9, 0.9, 0.9, 0.9, 0.88, 0.88, 0.87, 0.87, 0.86, 0.86, 0.85, 0.85, 0.85, 0.85, 0.85, 0.84, 0.84, 0.84, 0.84, 0.82, 0.8, 0.8, 0.8, 0.78, 0.78, 0.78, 0.77, 0.77, 0.77, 0.76, 0.76, 0.76, 0.75, 0.72, 0.72, 0.71, 0.66, 0.66, 0.66, 0.66]
[2025-03-24 00:19:48]  there are 138 items in inventory with total value: 207.19$
[2025-03-24 00:20:11]  bet_value = 2.21 -> bet_items

KeyboardInterrupt: 

In [11]:
press_no_doubledown_button()
press_accept_winnings_button_site()
accept_steam_trade(deposit=False, acceptWinning=True)

In [67]:
driver.switch_to.window(driver.window_handles[0])

In [65]:
log_bet(2.9, 
        3.22,
        1, 
        3.22, 
        203.97+3.22)